In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/patches/1/cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
!dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
!dpkg -i cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
!ls

In [ ]:
!nvidia-smi

Wed Jul  8 20:27:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    32W / 250W |    741MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

train_dataset = datasets.MNIST('./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST('./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

bs = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
class VAE(nn.Module):
  def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
    super(VAE, self).__init__()
    #encoder
    self.f1 = nn.Linear(x_dim, h_dim1)
    self.f2 = nn.Linear(h_dim1, h_dim2)
    self.f3_1 = nn.Linear(h_dim2, z_dim)
    self.f3_2 = nn.Linear(h_dim2, z_dim)
    #decoder
    self.f4 = nn.Linear(z_dim, h_dim2)
    self.f5 = nn.Linear(h_dim2, h_dim1)
    self.f6 = nn.Linear(h_dim1, x_dim)

  def encoder(self, x):
    h = F.relu(self.f1(x))
    h = F.relu(self.f2(h))
    return self.f3_1(h), self.f3_2(h) # mu, log_var
    
  def sampling(self, mu, log_var):
    std = torch.exp(0.5*log_var)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu) # return z sample
        
  def decoder(self, z):
    h = F.relu(self.f4(z))
    h = F.relu(self.f5(h))
    return F.sigmoid(self.f6(h)) 
    
  def forward(self, x):
      mu, log_var = self.encoder(x.view(-1, 784))
      z = self.sampling(mu, log_var)
      return self.decoder(z), mu, log_var

vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [ ]:
vae

VAE(
  (f1): Linear(in_features=784, out_features=512, bias=True)
  (f2): Linear(in_features=512, out_features=256, bias=True)
  (f3_1): Linear(in_features=256, out_features=2, bias=True)
  (f3_2): Linear(in_features=256, out_features=2, bias=True)
  (f4): Linear(in_features=2, out_features=256, bias=True)
  (f5): Linear(in_features=256, out_features=512, bias=True)
  (f6): Linear(in_features=512, out_features=784, bias=True)
)

In [ ]:
optimizer = optim.Adam(vae.parameters())
#loss functions
def loss_function(recon_x, x, mu, log_var):
  recon_loss = F.binary_cross_entropy(recon_x, x.view(-1, 28*28), reduction='sum')
  KL = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
  return recon_loss + KL


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(1, 51):
  train(epoch)
  test()

In [ ]:
torch.save(vae.state_dict(), '/content/gdrive/My Drive/checkpoint.pth')
state_dict = torch.load('/content/gdrive/My Drive/checkpoint.pth')
print(state_dict.keys())

odict_keys(['f1.weight', 'f1.bias', 'f2.weight', 'f2.bias', 'f3_1.weight', 'f3_1.bias', 'f3_2.weight', 'f3_2.bias', 'f4.weight', 'f4.bias', 'f5.weight', 'f5.bias', 'f6.weight', 'f6.bias'])


In [ ]:
vae.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), '/content/gdrive/My Drive/samples/sample5' + '.png')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
